# Forest: Tree crown detection

## Context
### Purpose
Detect tree crown using state-of-art models

### Modelling approach
A prebuilt Deep Learning model, named _DeepForest_, is used to predict individual tree crowns from an airborne RGB image.

### Highlights
* _DeepForest_ was trained on data from the National Ecological Observatory Network.

### Authors
Alejandro Coca-Castro, The Alan Turing Institute, [@acocac](https://github.com/acocac)

:::{note}
The author acknowledges [DeepForest](https://deepforest.readthedocs.io/en/latest/) contributors. Some code snippets were extracted from DeepForest [GitHub public repository](https://github.com/weecology/DeepForest).
:::

## Install and load libraries

In [1]:
!pip -q install git+https://github.com/ESM-VFC/intake_zenodo_fetcher.git ##Intake Zenodo Fetcher
!pip -q install DeepForest

In [2]:
import glob
import os
import urllib

import intake
from intake_zenodo_fetcher import download_zenodo_files_for_entry
import matplotlib.pyplot as plt
import xmltodict
import cv2

import tempfile

%matplotlib inline

## Fetch a RGB image from Zenodo

In [3]:
# create a temp dir
path = tempfile.mkdtemp()

catalog_file = os.path.join(path, 'catalog.yaml')

with open(catalog_file, 'w') as f:
    f.write('''
sources:
  NEONTREE_rgb:
    driver: xarray_image
    description: 'NeonTreeEvaluation RGB images (collection)'
    metadata:
      zenodo_doi: "10.5281/zenodo.3459803"
    args:
      urlpath: "{{ CATALOG_DIR }}/NEONsample_RGB/2018_MLBS_3_541000_4140000_image_crop.tif"
      ''')

Overwriting tc.yaml


In [4]:
cat_tc = intake.open_catalog('tc.yaml')

In [5]:
for catalog_entry in list(cat_tc):
    download_zenodo_files_for_entry(
        cat_tc[catalog_entry],
        force_download=False
    )

In [6]:
tc_rgb = cat_tc["NEONTREE_rgb"].to_dask()

/Users/ac/anaconda3/envs/envsensors/lib/python3.8/site-packages/intake_xarray/image.py:337: FutureWarning: open_files is deprecated and will be removed in a future release. Please use fsspec.core.open_files instead.
  files = open_files(self.urlpath, **self.storage_options)
/Users/ac/anaconda3/envs/envsensors/lib/python3.8/site-packages/xarray/core/dataset.py:2036: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  warnings.warn(


In [7]:
tc_rgb

<xarray.DataArray (y: 1864, x: 1429, channel: 3)>
dask.array<xarray-<this-array>, shape=(1864, 1429, 3), dtype=uint8, chunksize=(1864, 1429, 3), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) int64 0 1 2 3 4 5 6 7 ... 1857 1858 1859 1860 1861 1862 1863
  * x        (x) int64 0 1 2 3 4 5 6 7 ... 1422 1423 1424 1425 1426 1427 1428
  * channel  (channel) int64 0 1 2

## Load and prepare labels

In [8]:
filenames = glob.glob('./NEONsample_RGB/*.tif')
filesn = [os.path.basename(i) for i in filenames]

In [9]:
##Create ordered dictionary of .xml annotation files
def loadxml(imagename):
  imagename = imagename.replace('.tif','')
  fullurl = "https://raw.githubusercontent.com/weecology/NeonTreeEvaluation/master/annotations/" + imagename + ".xml"
  file = urllib.request.urlopen(fullurl)
  data = file.read()
  file.close()
  data = xmltodict.parse(data)
  return data

allxml = [loadxml(i) for i in filesn]


In [ ]:
# function to extract bounding boxes
def extractbb(i):
  bb = [f['bndbox'] for f in allxml[i]['annotation']['object']]
  return bb

bball = [extractbb(i) for i in range(0,len(allxml))]
print(len(bball))

## Visualise image and labels

In [ ]:
# function to plot images
def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)

    return plt.imshow(a, **kwargs)

In [ ]:
image = tc_rgb

In [ ]:
# plot predicted bbox
image2 = image.values.copy()
target_bbox = bball[0]

for row in target_bbox:
    cv2.rectangle(image2, (int(row["xmin"]), int(row["ymin"])), (int(row["xmax"]), int(row["ymax"])), (0, 0, 0), thickness=10, lineType=cv2.LINE_AA)

plt.figure(figsize=(15,15))
cv2_imshow(image2)
plt.show()

## Load *DeepForest* pretrained model

In [ ]:
from deepforest import main

In [ ]:
# load deep forest model
model = main.deepforest()
model.use_release()

In [ ]:
#boxes = model.predict_image(image=image.values)
#print(boxes)